# Lecture 3: Password Attack with Differential-Power-Analysis (Kocher et al. 1999)

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import numpy as np
import plotly.graph_objects as pgo
import lascar

from securec.capture import capture

In [ ]:
data = capture(
    platform="elmo",
    number_of_traces=1000,
    number_of_samples=0,
    fromfile=os.path.abspath("sbox_lookup.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)],
)

In [ ]:
def selection_function(value, guess):
    return lascar.tools.aes.sbox[value["input"][0] ^ guess] & 0x01 == 0


trace = lascar.TraceBatchContainer(data["trace"], data)
engine = lascar.DpaEngine(
    "dpa",
    selection_function=selection_function,
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=lascar.TableOutputMethod(engine),
)
session.run(batch_size="auto")
